In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data

import torchvision.transforms as tfs
from torchvision.models import resnet50, ResNet50_Weights

import random

from src.YoloCigaretteDataset import get_pos_neg_files, YoloCigaretteDataset
from src.visualizeRandomSamples import visualize_random_samples

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [50]:
# Параметры (можно менять) 
TRAIN_IMG_DIR = "Dataset/train/images"
TRAIN_LABEL_DIR = "Dataset/train/labels"

VAL_IMG_DIR = "Dataset/val/images"
VAL_LABEL_DIR = "Dataset/val/labels"

# сколько хотим взять
TRAIN_POS_N = 2000
TRAIN_NEG_N = 2000

VAL_POS_N = 250
VAL_NEG_N = 250

BATCH_SIZE = 4
KEEP_CLASS = 0  # класс сигареты

# Получаем списки для train и val
train_pos_all, train_neg_all = get_pos_neg_files(TRAIN_IMG_DIR, TRAIN_LABEL_DIR, keep_class=KEEP_CLASS)
val_pos_all, val_neg_all     = get_pos_neg_files(VAL_IMG_DIR, VAL_LABEL_DIR, keep_class=KEEP_CLASS)

print("Всего доступных (train): pos =", len(train_pos_all), "neg =", len(train_neg_all))
print("Всего доступных (val):   pos =", len(val_pos_all),   "neg =", len(val_neg_all))

# Перемешиваем и режем по нужному кол-ву (по факту можно добавить проверку на длину)
random.shuffle(train_pos_all)
random.shuffle(train_neg_all)
random.shuffle(val_pos_all)
random.shuffle(val_neg_all)

train_pos = train_pos_all[:TRAIN_POS_N]
train_neg = train_neg_all[:TRAIN_NEG_N]

val_pos = val_pos_all[:VAL_POS_N]
val_neg = val_neg_all[:VAL_NEG_N]

train_files = train_pos + train_neg
val_files = val_pos + val_neg

random.shuffle(train_files)
random.shuffle(val_files)

print("Финально для train взято:", len(train_files), "(pos,neg) =", len(train_pos), len(train_neg))
print("Финально для val   взято:", len(val_files),   "(pos,neg) =", len(val_pos),   len(val_neg))

transforms_train = tfs.Compose([
    tfs.RandomResizedCrop(64, scale=(0.8, 1.0)),  
    tfs.RandomHorizontalFlip(p=0.5),
    tfs.RandomRotation(10),
    tfs.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    tfs.ToTensor(),
    tfs.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

transforms_val = tfs.Compose([
    tfs.ToTensor(),
    tfs.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

#Создаём датасеты и лоадеры 
train_dataset = YoloCigaretteDataset(TRAIN_IMG_DIR, TRAIN_LABEL_DIR, train_files, transform=transforms_train, keep_class=KEEP_CLASS, RCNN=True)
val_dataset   = YoloCigaretteDataset(VAL_IMG_DIR,   VAL_LABEL_DIR,   val_files,   transform=transforms_val, keep_class=KEEP_CLASS, RCNN=True)

train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,  collate_fn=lambda batch: tuple(zip(*batch)))
val_loader   = data.DataLoader(val_dataset,   batch_size=BATCH_SIZE, shuffle=False, collate_fn=lambda batch: tuple(zip(*batch)))


Всего доступных (train): pos = 5088 neg = 219
Всего доступных (val):   pos = 257 neg = 13
Финально для train взято: 2219 (pos,neg) = 2000 219
Финально для val   взято: 263 (pos,neg) = 250 13


## Бэкбон модели - глаз модели

In [30]:
class backbone(nn.Module):
    
    def __init__(self, pretrained=True, trainable_layers=2):
        super().__init__()
        weights = ResNet50_Weights.IMAGENET1K_V2 if pretrained else None # подгружаем весас ResNet50, обученные на ImageNet
        r = resnet50(weights=weights) # Задаем веса модели ResNet
        
        self.body = nn.Sequential( # Берем всю внутрянку, без классификатора - нам нужен только расспознаватель образов
            r.conv1, 
            r.bn1,
            r.relu,
            r.maxpool,
            r.layer1,
            r.layer2,
            r.layer3,
            r.layer4
        )
        
        self._freeze(r, trainable_layers=trainable_layers) # Включаем обучение части слоев
        
        self.out_channels = 2048 # Выходное кол-во каналов
        
    def _freeze(self, r, trainable_layers=2):
        
        for p in r.parameters():
            p.requires_grad = False # Морозит градиенты на всех слоях
            
        layers = [r.layer1, r.layer2, r.layer3, r.layer4]
        for l in layers[-trainable_layers]:
            for p in l.parameters():
                p.requires_grad = True
                
    def forward(self, x):
        return self.body(x)
    

## Генератор якорей (acnhor) - раскладываем якоря по карте признаков

In [31]:
from torchvision.models.detection.rpn import AnchorGenerator, RPNHead, RegionProposalNetwork
from torchvision.models.detection.image_list import ImageList
from torchvision.ops import MultiScaleRoIAlign
from torchvision.models.detection.roi_heads import RoIHeads
from torchvision.models.detection.faster_rcnn import TwoMLPHead, FastRCNNPredictor


class MyFasterRCNN(nn.Module):
    
    def __init__(self, num_classes, roi_size=7):
        super().__init__()
        self.backbone = backbone()
        self.archor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),), # Кортеж кортежей потому, что карт признаков может быть несколько, если мы пытаемся обобщать масштаб
                                            aspect_ratios=((0.5, 1.0, 2.0),))
        self.rpn_head = RPNHead(in_channels=self.backbone.out_channels, num_anchors=self.archor_generator.num_anchors_per_location()[0])
        self.rpn = RegionProposalNetwork(
            anchor_generator=self.archor_generator,
            head=self.rpn_head,
            fg_iou_thresh=0.7, # Порог пересечения, выше которого якорь считается позитивным
            bg_iou_thresh=0.3, # Порог негативного якоря
            batch_size_per_image=256, # сколько якорей идет в батч для вычисления loss
            positive_fraction=0.5, # Такая доля из батча должна быть позитивной, чтобы сеть быстрее научилась определять фон
            pre_nms_top_n={"training": 1000, "testing": 500}, # Сколько лучших боксов оставить до NMS
            post_nms_top_n={"training": 300, "testing": 300}, # Сколько боксов оставить после NMS
            nms_thresh=0.7, # Если между боксами IoU более 0.7, то слабый бокс будет отброшен и будет считаться частью сильного
            score_thresh=0.0
        )

        self.roi_size = roi_size
        self.box_roi_pool = MultiScaleRoIAlign(
            featmap_names=['0'],
            output_size=self.roi_size,
            sampling_ratio=2
        ) # Вырезаем соотвествующую область на карте признаков, приводим к фикс. размеру
        
        self.num_classes = num_classes
        
        representation_size = 1024
        box_head_in = self.backbone.out_channels * roi_size * roi_size
        self.box_head = TwoMLPHead(in_channels=box_head_in, representation_size=representation_size) # Два полносвязных слоя
        self.box_predictor = FastRCNNPredictor(in_channels=representation_size, num_classes=num_classes) # Решение задачи классификкации, задачи регрессии
        
        
        self.roi_heads = RoIHeads(
            box_roi_pool=self.box_roi_pool,
            box_head=self.box_head,
            box_predictor=self.box_predictor,
            fg_iou_thresh=0.5,
            bg_iou_thresh=0.5,
            batch_size_per_image=256,
            positive_fraction=0.25,
            bbox_reg_weights=None,
            score_thresh=0.05,
            nms_thresh=0.5,
            detections_per_img=100,
        )

    def forward(self, x, targets=None):

        image_sizes = [img.shape[-2:] for img in x] # Сохранение размерностей всех картинок
        batch_tensor = self.pad_images_to_max_size(x) # Приведение картинок к общему размеру по максимальному в батче

        image_list = ImageList(batch_tensor, image_sizes) # Каждому изображению новой размерности соотвествует его старая размерность

        feature_map = self.backbone(image_list.tensors) # Прогнали картинки через backbone и получили на выходе карту признаков
        features = {'0': feature_map}
        
        proposals, rpn_losses = self.rpn(image_list, features, targets) # выделение потенциальных регионов, генерация якорей разных рамеров и пропорций, происходит отбор наиболее потенциальных якорей
        detections, roi_losses = self.roi_heads(features, proposals, image_sizes, targets)
        
        if self.training:
            losses = {}
            losses.update(rpn_losses)
            losses.update(roi_losses)
            
            return losses
        
        return detections
    
    @staticmethod
    def pad_images_to_max_size(x, pad_val=0):
        max_h = max(i.shape[-2] for i in x)
        max_w = max(i.shape[-1] for i in x)
        
        batch = []
        for i in x:
            c, h, w = i.shape
            padded = i.new_full((c, max_h, max_w), pad_val)
            padded[:, :h, :w] = i
            batch.append(padded)
        
        return torch.stack(batch)
    
    

# Модель PyTorch

In [51]:
import torch
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN_ResNet50_FPN_Weights

num_classes = 2  # фон + сигарета

weights = FasterRCNN_ResNet50_FPN_Weights.DEFAULT
model = fasterrcnn_resnet50_fpn(weights=weights)

in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

for p in model.backbone.parameters():
    p.requires_grad = False # морожу все параметры / отключаю обучение
    
for p in model.backbone.body.layer3.parameters():
    p.requires_grad = True
for p in model.backbone.body.layer4.parameters():
    p.requires_grad = True
    
model.transform.min_size = (480, )
model.transform.max_size = 800

model.rpn.post_nms_top_n_train = 300
model.rpn.post_nms_top_n_test = 300

model.rpn.pre_nms_top_n_train = 1000
model.rpn.pre_nms_top_n_test = 500

model.roi_heads.batch_size_per_image = 128
model.roi_heads.positive_fraction = 0.25

In [52]:
from tqdm.auto import tqdm
from torchmetrics.detection.mean_ap import MeanAveragePrecision
metric = MeanAveragePrecision(iou_thresholds=[0.5]).to(device)

# model = MyFasterRCNN(2)

model = model.to(device)

epochs = 10
optimizer = torch.optim.AdamW(params=[p for p in model.parameters() if p.requires_grad], lr=0.001, weight_decay=0.001)

best_map = 0
for epoch in range(epochs):
    
    metric.reset()
    
    train_bar = tqdm(train_loader, desc=f'Эпоха тренировочная {epoch+1}/{epochs}', position=0)
    
    model.train()
    for x_train, y_train in train_bar:
        x_train = [x.to(device) for x in x_train]
        y_train = [{k: v.to(device) for k, v in t.items()} for t in y_train]
        
        loss = sum(model(x_train, y_train).values())

        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_bar.set_postfix({
            'loss': loss
        })

        
    model.eval()
    
    val_bar = tqdm(val_loader, desc=f'Эпоха валидационная {epoch+1}/{epochs}', position=0)
    for x_val, y_val in val_bar:
        
        x_val = [x.to(device) for x in x_val]
        y_val = [{k: v.to(device) for k, v in t.items()} for t in y_val]
        
        with torch.no_grad():
            pred = model(x_val)
            
        metric.update(pred, y_val)
        res = metric.compute()
        _map = float(res['map_50'].item())
        
        val_bar.set_postfix({
            'map': _map
        })

    
    if _map >= best_map:
        print(f'Сохранена модел с результатом MAP: {_map}')
        best_map = _map
        torch.save(model.state_dict(), 'best_model_frcnn.tar')





Эпоха валидационная 1/10: 100%|██████████| 66/66 [01:55<00:00,  1.75s/it, map=1.26e-5] 


Сохранена модел с результатом MAP: 1.2628814147319645e-05


Эпоха валидационная 3/10: 100%|██████████| 66/66 [00:57<00:00,  1.14it/s, map=5.15e-5] 


Сохранена модел с результатом MAP: 5.147828414919786e-05


Эпоха валидационная 4/10: 100%|██████████| 66/66 [01:01<00:00,  1.08it/s, map=8.39e-5] 


Сохранена модел с результатом MAP: 8.390149014303461e-05


Эпоха валидационная 10/10: 100%|██████████| 66/66 [01:13<00:00,  1.11s/it, map=0]
